In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
import tensorflow
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.models import Sequential

In [ ]:
nltk.download('all')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Exercise 1: Fake News Classification

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Datasets/train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
df = df.dropna()

In [ ]:
X = df['title'] + " " + df['text']
y = df['label']

In [ ]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

In [ ]:
corpus = []
for text in X:
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    text = text.lower()
    words = word_tokenize(text)
    words = [word for word in words if word not in stop_words]
    corpus.append(' '.join(words))

In [ ]:
max_words = 10000
tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')
tokenizer.fit_on_texts(corpus)
X_sequences = tokenizer.texts_to_sequences(corpus)
X_padded = pad_sequences(X_sequences, maxlen=200)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.2, random_state=42)

In [ ]:
embedding_dim = 100
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=X_padded.shape[1]))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 100)          1000000   
                                                                 
 lstm (LSTM)                 (None, 128)               117248    
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1117377 (4.26 MB)
Trainable params: 1117377 (4.26 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=64)

Epoch 1/5
229/229 [==============================] - 52s 192ms/step - loss: 0.2673 - accuracy: 0.8830 - val_loss: 0.1677 - val_accuracy: 0.9418
Epoch 2/5
229/229 [==============================] - 25s 109ms/step - loss: 0.1106 - accuracy: 0.9632 - val_loss: 0.1750 - val_accuracy: 0.9357
Epoch 3/5
229/229 [==============================] - 15s 66ms/step - loss: 0.0587 - accuracy: 0.9814 - val_loss: 0.1949 - val_accuracy: 0.9341
Epoch 4/5
229/229 [==============================] - 13s 57ms/step - loss: 0.0311 - accuracy: 0.9900 - val_loss: 0.2442 - val_accuracy: 0.9103
Epoch 5/5
229/229 [==============================] - 8s 33ms/step - loss: 0.0347 - accuracy: 0.9885 - val_loss: 0.2242 - val_accuracy: 0.9278


In [ ]:
y_pred = (model.predict(X_test) > 0.5).astype(int)

115/115 [==============================] - 1s 5ms/step


In [ ]:
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("Accuracy:", 100*accuracy_score(y_test, y_pred))

Confusion Matrix:
[[1942  140]
 [ 124 1451]]
Accuracy: 92.78096800656276


The accuracy for Fake News detections is 92.78%

##Exercise 2: Fake News Classification for Hindi Dataset

In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/Datasets/Hindi_News/fake_news_basic.csv',index_col=0)
df2 = pd.read_csv('/content/drive/MyDrive/Datasets/Hindi_News/true_news_basic.csv',index_col=0)
df1['label'] = 1
df2['label'] = 0
df=pd.concat([df1,df2],ignore_index=True)
df.head()

,short_description,label
0,बूम पाय इमरान खान भारत सरकार आलोच रह वर्तमान श...,1
1,सिख समुदाय के लोग हिंद साइन बोर्ड कालिख पोत दि...,1
2,सोशल मीडिय प्लेटफ़ॉर्म फ़ेसबुक ट्विटर दाव के बड़ ...,1
3,दाव भाजप के मा सरकार जन के गलत रह ।,1
4,मीडिय आउटलेट्स वायर एजेंस गलत तरीक दाव किय पाक...,1


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2010 entries, 0 to 2009
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   short_description  2010 non-null   object
 1   label              2010 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 31.5+ KB


In [ ]:
df['label'].value_counts()

1    1250
0     760
Name: label, dtype: int64

In [ ]:
X = df['short_description']
y = df['label']

In [ ]:
with open('/content/drive/MyDrive/Datasets/Hindi_News/final_stopwords.txt', 'r', encoding='utf-8') as file:
    stop_words = [line.strip() for line in file]

corpus = []
for text in X:
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    text = text.lower()
    words = word_tokenize(text)
    words = [word for word in words if word not in stop_words]
    corpus.append(' '.join(words))

In [ ]:
max_words = 10000
tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')
tokenizer.fit_on_texts(corpus)
X_sequences = tokenizer.texts_to_sequences(corpus)
X_padded = pad_sequences(X_sequences, maxlen=200)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.2, random_state=1)

In [ ]:
embedding_dim = 100
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=X_padded.shape[1]))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 200, 100)          1000000   
                                                                 
 lstm_1 (LSTM)               (None, 128)               117248    
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1117377 (4.26 MB)
Trainable params: 1117377 (4.26 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=16)  # Adjust epochs and batch size

Epoch 1/5
101/101 [==============================] - 6s 34ms/step - loss: 0.6691 - accuracy: 0.6157 - val_loss: 0.6679 - val_accuracy: 0.6119
Epoch 2/5
101/101 [==============================] - 1s 13ms/step - loss: 0.6639 - accuracy: 0.6244 - val_loss: 0.6681 - val_accuracy: 0.6119
Epoch 3/5
101/101 [==============================] - 1s 13ms/step - loss: 0.6638 - accuracy: 0.6244 - val_loss: 0.6682 - val_accuracy: 0.6119
Epoch 4/5
101/101 [==============================] - 1s 11ms/step - loss: 0.6627 - accuracy: 0.6244 - val_loss: 0.6696 - val_accuracy: 0.6119
Epoch 5/5
101/101 [==============================] - 2s 18ms/step - loss: 0.6631 - accuracy: 0.6244 - val_loss: 0.6703 - val_accuracy: 0.6119


In [ ]:
y_pred = (model.predict(X_test) > 0.5).astype(int)

13/13 [==============================] - 1s 6ms/step


In [ ]:
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("Accuracy:", 100*accuracy_score(y_test, y_pred))

Confusion Matrix:
[[  0 156]
 [  0 246]]
Accuracy: 61.19402985074627


The accuracy for Hindi Fake News detections is 61.19%